In [164]:
import time
import os
from urllib.request import urlopen
from selenium import webdriver # allow launching browser
from selenium.webdriver.common.by import By # allow search with parameters
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from selenium.webdriver.support import expected_conditions as EC # determine whether the web page has loaded
from selenium.common.exceptions import TimeoutException # handling timeout situation
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
from datetime import date 

In [165]:
driver_option = webdriver.ChromeOptions()
driver_option.add_argument('--headless')
# driver_option.add_argument('--incognito')
chromedriver_path = '/Users/ellpeeaxe/Desktop/SMU/Y4S1/SA/chromedriver' # Change this to your own chromedriver path!

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

In [166]:
def get_posturls(username):
    
    browser = create_webdriver()
    # Open the website
    browser.get('https://www.instagram.com/'+username)

    # Get the number of posts
    post_num_element = browser.find_element_by_class_name('g47SY')
    post_num_html = post_num_element.get_attribute('innerHTML')

    # Retrieve number of posts
    soup = BeautifulSoup(post_num_html, 'html.parser')
    num_posts = int(soup.get_text().replace(',', ''))


    urls = []

    body = browser.find_element_by_css_selector('body')

    posts_element = browser.find_element_by_class_name('_2z6nI')
    posts_html = posts_element.get_attribute('innerHTML')

    soup = BeautifulSoup(posts_html, 'html.parser')
    posts = soup.find_all('div', {'class': 'v1Nh3 kIKUG _bz0w'})

    for post in posts:
        urls.append(post.find('a')['href'])
        
    print(username + " " + str(len(urls)))

    browser.close()

    return urls

In [167]:
def scrape_likescomments(urls):
    posts = []
    likes = []
    comment_count = []
    commenters = []
    comments = []

    for url in list(urls):
        url = 'http://instagram.com' + url
        print(url)
        page = urlopen(url).read()
        soup = BeautifulSoup(page, features="html.parser")

        string = soup.find("meta",  property="og:description")['content']
        string = string.split(" ")
        num_likes = string[0]
        num_comments = string[2]

        browser = create_webdriver()
        browser.get(url)

        hasLoadMore = True
        while hasLoadMore:
            time.sleep(2)
            try:
                browser.find_element_by_css_selector('button.dCJp8.afkep > span.glyphsSpriteCircle_add__outline__24__grey_9.u-__7').click()
            except:
                hasLoadMore = False

        texts = browser.find_elements_by_class_name('C4VMK')
       
        for comment in texts:
            posts.append(url)
            likes.append(num_likes)
            comment_count.append(num_comments)
            try:
                comments.append(comment.text.split("\n")[1])
                commenters.append(comment.text.split("\n")[0])
            except:
                commenters.append(" ")
                comments.append(" ")

        browser.close()

    df = pd.DataFrame({'url': posts, 
                       'num_likes': likes, 
                       'num_comment': comment_count, 
                       'commenter': commenters, 
                       'comment': comments})
    return df
    

In [168]:
if not os.path.exists(os.getcwd()+"/data/LikesComments/"):
    os.makedirs(os.getcwd()+"/data/LikesComments/")
    
for username in open("usernames.txt", "r"):
    urls = get_posturls(username.strip())
    df = scrape_likescomments(urls)
    df['username'] = username.strip()
    
    display(df)
    
    df.to_csv(os.getcwd()+"/data/LikesComments/"+username.strip()+"-"+str(date.today())+'.txt',sep='\t',index = False,encoding='utf-8')


/Users/ellpeeaxe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


juin66 12
http://instagram.com/p/B2n9QpkgkuS/
http://instagram.com/p/B2gCFN7g1cS/
http://instagram.com/p/B2UA20jA--Y/
http://instagram.com/p/B2N0RLdAU35/
http://instagram.com/p/B2D03RiAhHS/
http://instagram.com/p/B10RKPIgcVB/
http://instagram.com/p/B1yIx-Ogm2T/
http://instagram.com/p/B1sudHmgzTz/
http://instagram.com/p/B1kh98kAtgQ/
http://instagram.com/p/B1YOwhRA8wm/
http://instagram.com/p/B1RgKE_AUDj/
http://instagram.com/p/B1OWv4wAgX2/


,comment,commenter,num_comment,num_likes,url,username
0,,,6,96,http://instagram.com/p/B2n9QpkgkuS/,juin66
1,,,6,96,http://instagram.com/p/B2n9QpkgkuS/,juin66
2,,,6,96,http://instagram.com/p/B2n9QpkgkuS/,juin66
3,,,6,96,http://instagram.com/p/B2n9QpkgkuS/,juin66
4,我常到餐厅想点🥛一杯热水,juin66,5,237,http://instagram.com/p/B2gCFN7g1cS/,juin66
5,这个星期天 Bugis+ 1pm-6pm,juin66,11,191,http://instagram.com/p/B2UA20jA--Y/,juin66
6,太可惜不能去😭,_yamalilyryo,11,191,http://instagram.com/p/B2UA20jA--Y/,juin66
7,我要玩那个 cream 的,ohyushi,11,191,http://instagram.com/p/B2UA20jA--Y/,juin66
8,"If I have anything useless, can I bring along ...",vinc_entertain_vincent,11,191,http://instagram.com/p/B2UA20jA--Y/,juin66
9,@juin66 明天见哦希望明天可以和你换到礼物❤️❤️❤️❤️❤️❤️❤️❤️❤️,exmund,11,191,http://instagram.com/p/B2UA20jA--Y/,juin66


vaneszs.h 12
http://instagram.com/p/B2tiNaBJ_jm/
http://instagram.com/p/B2q2DM9JdzF/
http://instagram.com/p/B2oW3ijpkG-/
http://instagram.com/p/B2jOLJ4pFxR/
http://instagram.com/p/B2eUdANpVJQ/
http://instagram.com/p/B2bf27qpXcC/
http://instagram.com/p/B2Y54RIJ8BD/
http://instagram.com/p/B2T4zEnJwMa/
http://instagram.com/p/B2JbPU-Hjol/
http://instagram.com/p/B2HMcEop5k1/
http://instagram.com/p/B2EcBYRpaO6/
http://instagram.com/p/B1_HOFRJyM6/


,comment,commenter,num_comment,num_likes,url,username
0,Exciting time of the year again! 😃 Thanks @aid...,vaneszs.h,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
1,😍😍😍👍🏻,tylomere,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
2,Bestest plus one 😛😛,aidsinthe,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
3,I still have to experience it live! Maybe one ...,ralph.cardinale,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
4,So elegant ❤️❤️❤️,ramjanali.shah.902266,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
5,😘,agunk_to9,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
6,😘😘,tingting__89,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
7,Nice very nice ❤️❤️😍😍👍👍👍🙋‍♂️,dt533777,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
8,The fog setting on this pic must’ve been set to 0,aberhamzah,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h
9,😍😍😍beautiful stunning gorgeous,munz38,16,"2,852",http://instagram.com/p/B2tiNaBJ_jm/,vaneszs.h


...

,comment,commenter,num_comment,num_likes,url,username
0,Global warming is the greatest environmental t...,tylertennn,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
1,Save me too,evolsivart,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
2,☹♻️🌎,huipingfoo,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
3,but you the only causing the global warming ho...,andynation_,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
4,💗👍🏽,gail_boj,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
5,😍,snow_sa2109,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
6,👍,tingting__89,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
7,❤️❤️❤️❤️lovely smile,cal81007,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
8,👍,tingting__89,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn
9,❤️❤️❤️❤️lovely smile,cal81007,7,393,http://instagram.com/p/B2eB9DbHB1u/,tylertennn


zane_lim 12
http://instagram.com/p/B2tkYBwHIDs/
http://instagram.com/p/B2oTq1Incch/
http://instagram.com/p/B2eHpPlH4XX/
http://instagram.com/p/B18lTKXnN0S/
http://instagram.com/p/B13Yob3nHAZ/
http://instagram.com/p/B1q0CVtnTVN/
http://instagram.com/p/B1bD54GJvGF/
http://instagram.com/p/B1L2J7sHYN_/
http://instagram.com/p/B1ERsudnWYz/
http://instagram.com/p/B0TDAeEHSBX/
http://instagram.com/p/B0Nus0wntyK/
http://instagram.com/p/Bz-Xe9xnbDn/


,comment,commenter,num_comment,num_likes,url,username
0,“Do not wait; the time will never be ‘just rig...,zane_lim,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
1,Your white pants not dirty ah,glenthez,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
2,@glenthez nopeeeeee 😂,zane_lim,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
3,My inspiration❤️,iamhermankeh,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
4,@iamhermankeh You are my everything ❤️❤️,zane_lim,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
5,Wow r u my 白马王子??😘😘,tingting__89,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
6,Nice 🤙🏼,wen_kang_,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
7,Wow I’m your biggest fan !!!😍,clarencetayyy,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
8,@clarencetayyy same heree!! 😆😆😆,zane_lim,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim
9,love you @zane_lim ❤️❤️ please reply,faithkohh,21,571,http://instagram.com/p/B2tkYBwHIDs/,zane_lim


bryaa.n 12
http://instagram.com/p/B2n6TgzgAL0/
http://instagram.com/p/B1jV55igaW8/
http://instagram.com/p/B1T4aq2gry1/
http://instagram.com/p/B1B9ztbgn8c/
http://instagram.com/p/B0bAd4cgXWp/
http://instagram.com/p/B0OCfEwAm-5/
http://instagram.com/p/Bz8a5bEgRRz/
http://instagram.com/p/BziI_WTgj1p/
http://instagram.com/p/BzgKXf7ALVk/
http://instagram.com/p/Byet3YZgKU2/
http://instagram.com/p/BdFhBMXFRS4/
http://instagram.com/p/qWLTrhGtJ7/


IncompleteRead: IncompleteRead(11991 bytes read, 43776 more expected)

In [172]:
username = "iamjoel.low"
urls = get_posturls(username.strip())
df = scrape_likescomments(urls)
df['username'] = username.strip()

display(df)

df.to_csv(os.getcwd()+"/data/LikesComments/"+username.strip()+"-"+str(date.today())+'.txt',sep='\t',index = False,encoding='utf-8')



/Users/ellpeeaxe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


iamjoel.low 12
http://instagram.com/p/B2hOGaWnGoS/
http://instagram.com/p/B2cCXbfnN4m/
http://instagram.com/p/B2ZXQ8UnUsa/
http://instagram.com/p/B2ZPg5Sn_aN/
http://instagram.com/p/B2UU6Lvn1Ew/
http://instagram.com/p/B2URuobnLYb/
http://instagram.com/p/B2RoFb_nGkJ/
http://instagram.com/p/B2PAYqtHuL4/
http://instagram.com/p/B2JeVVaHCSj/
http://instagram.com/p/B2EFDpSnvR8/
http://instagram.com/p/B1-kUTunfwJ/
http://instagram.com/p/B155LB_nGie/


,comment,commenter,num_comment,num_likes,url,username
0,,,0,38,http://instagram.com/p/B2hOGaWnGoS/,iamjoel.low
1,感谢今天前来支持我们的朋友！💕,iamjoel.low,8,144,http://instagram.com/p/B2cCXbfnN4m/,iamjoel.low
2,Spent a lovely afternoon together with Emily o...,iamjoel.low,2,78,http://instagram.com/p/B2ZXQ8UnUsa/,iamjoel.low
3,"If you believe you can, you will make it! Have...",jsjoycestella,2,78,http://instagram.com/p/B2ZXQ8UnUsa/,iamjoel.low
4,❤️❤️😂😍,t4an4duy,2,78,http://instagram.com/p/B2ZXQ8UnUsa/,iamjoel.low
5,最近的空气质量实在不太好！😥 如果大家出门的话，一定要记得戴上口罩、保护自己！😷 我也希望大...,iamjoel.low,1,57,http://instagram.com/p/B2ZPg5Sn_aN/,iamjoel.low
6,💪💪💪,tingting__89,1,57,http://instagram.com/p/B2ZPg5Sn_aN/,iamjoel.low
7,,,0,41,http://instagram.com/p/B2UU6Lvn1Ew/,iamjoel.low
8,今晚看了《天气之子》之后感触良多。戏中的男主角为了逃离平凡的乡下日子、自己一人跑去东京讨个生...,iamjoel.low,1,73,http://instagram.com/p/B2URuobnLYb/,iamjoel.low
9,Sometimes loneliness and sadness can creep sil...,iamjoel.low,2,80,http://instagram.com/p/B2RoFb_nGkJ/,iamjoel.low
